In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['HF_HOME'] = '/nfs/namsan/hf_cache'

In [2]:
from dataclasses import dataclass, field
from typing import Optional

import peft
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

/nfs/namsan/pyenv/versions/fm_teacher/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /nfs/namsan/pyenv/versions/fm_teacher/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


/nfs/namsan/pyenv/versions/fm_teacher/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/ssl/lib'), PosixPath('/usr/local/cuda/extras/CUPTI/lib64'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)


In [3]:
@dataclass
class ScriptArguments:
    model_name: str = './opt_750'

parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses([])[0]

In [4]:
peft_model_id = script_args.model_name
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    return_dict=True,
#     torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

In [5]:
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): OPTForCausalLM(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 2560, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 2560)
          (final_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0): OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
                (v_proj): Linear(
                  in_features=2560, out_features=2560, bias=True
                  (lora_dropout): Dropout(p=0.05, inplace=False)
                  (lora_A): Linear(in_features=2560, out_features=16, bias=False)
                  (lora_B): Linear(in_features=16, out_features=2560, bias=False)
                )
                (q_proj): Linear(
                  in_features=2560, out_features=2560, bias=True
             

In [6]:
key_list = [key for key, _ in model.base_model.model.named_modules() if "lora" not in key]
for key in key_list:
    parent, target, target_name = model.base_model._get_submodules(key)
    if isinstance(target, peft.tuners.lora.Linear):
        bias = target.bias is not None
        new_module = torch.nn.Linear(target.in_features, target.out_features, bias=bias)
        model.base_model._replace_module(parent, target_name, new_module, target)

model = model.base_model.model

In [7]:
model.save_pretrained("./opt_750_merged")